In [1]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [2]:
def switch_chude(var):
    switcher = {
                1: "data_final\\bạn bè.txt", 
                2: "data_final\\các câu hỏi phức tạp.txt", 
                3: "data_final\\đất nước.txt",
                4: "data_final\\địa chỉ.txt", 
                5: "data_final\\du lịch.txt", 
                6: "data_final\\gia đình.txt", 
                7: "data_final\\giải trí.txt", 
                8: "data_final\\học tập.txt", 
                9: "data_final\\nghề nghiệp.txt", 
                10: "data_final\\nghi lễ.txt", 
                11: "data_final\\người yêu.txt", 
                12: "data_final\\robot.txt", 
                13: "data_final\\shopping.txt", 
                14: "data_final\\sở thích.txt", 
                15: "data_final\\tán gẫu.txt", 
                16: "data_final\\tdtu.txt", 
                17: "data_final\\thông tin cá nhân.txt", 
                18: "data_final\\trò chuyện về đi ăn.txt"
    }

    return switcher.get(var)

In [3]:
print('1.Bạn bè   2.Các câu hỏi phức tạp   3.Đất nước   4.Địa chỉ   5.Du lịch   6.Gia đình')
print('7.Giải trí   8.Học tập   9.Nghề nghiệp   10.Nghi lễ   11.Người yêu   12.Robot')
print('13.Shopping   14.Sở thích   15.Tán ngẫu   16.TDTU   17.Thông tin cá nhân   18.Trò chuyện về đi ăn\n')

subject = int(input('Chọn chủ đề: '))
while subject<1 or subject>18:
    subject = int(input('Chọn lại chủ đề: '))
    
path = switch_chude(subject)

1.Bạn bè   2.Các câu hỏi phức tạp   3.Đất nước   4.Địa chỉ   5.Du lịch   6.Gia đình
7.Giải trí   8.Học tập   9.Nghề nghiệp   10.Nghi lễ   11.Người yêu   12.Robot
13.Shopping   14.Sở thích   15.Tán ngẫu   16.TDTU   17.Thông tin cá nhân   18.Trò chuyện về đi ăn

Chọn chủ đề: 15


In [4]:
list_sent=[]
with open(path, encoding="utf8") as f:
    train_lines = f.readlines()
    for line in train_lines:
        line = line.split('__eou__')
        line.remove('\n')
        for i in range(len(line)):
            line[i] = line[i].strip()
        list_sent.append(line)

get_Question = []
for i in list_sent:
    get_Question.append(i[0])

get_Answer = []
for i in list_sent:
    get_Answer.append(i[1])
    
labels = []
for i in range(len(get_Answer)):
    labels.append(i)

df = pd.DataFrame({"questions": get_Question, "answers": get_Answer, "label": labels})
df

,questions,answers,label
0,Bạn thích mình không?,Mình rất thích bạn,0
1,Bạn thích xem phim 18+ chứ?,Thỉnh thoảng mình vẫn thích,1
2,Cho mình 200 triệu được không?,Bạn đưa số tài khoản cho mình đi,2
3,Đi nhảy đầm với mình không?,"Ok, tôi nay hẹn bạn 10h",3
4,Trong bạn ngại ngùng thật đáng yêu?,Cảm ơn bạn,4
...,...,...,...
201,bạn đang làm gì đó?,Hỏi chi ?,201
202,này là cái gì vậy?,Này là cái gì ?,202
203,bạn có thích đồng tính không?,"Không, tui chơi gay",203
204,bạn có từng làm từ thiện chưa?,Rồi nha,204


In [5]:
num_classes = len(get_Question)

In [6]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(labels)
labels = lbl_encoder.transform(labels)

In [7]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# the size of the vocabulary in the text data
vocab_size = 2000
# size of the vector space in which words will be embedded
embedding_dim = 16
# length of input sequences
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(get_Question)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(get_Question)

# The “pad_sequences” method is used to make all the training text sequences into the same size
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [16]:
print(word_index)

{'<OOV>': 1, 'bạn': 2, 'không': 3, 'có': 4, 'đi': 5, 'chưa': 6, 'mày': 7, 'ăn': 8, 'thích': 9, 'mình': 10, 'với': 11, 'nào': 12, 'sao': 13, 'gì': 14, 'vậy': 15, 'hay': 16, 'cơm': 17, 'là': 18, 'em': 19, 'làm': 20, 'thế': 21, 'người': 22, 'chơi': 23, 'học': 24, 'khi': 25, 'về': 26, 'cho': 27, 'yêu': 28, 'cậu': 29, 'mai': 30, 'ai': 31, 'ra': 32, 'tao': 33, 'tiền': 34, 'rồi': 35, 'biết': 36, 'bao': 37, 'bị': 38, 'mua': 39, 'này': 40, 'tính': 41, 'cái': 42, 'một': 43, 'mới': 44, 'đang': 45, 'được': 46, 'ngủ': 47, 'thấy': 48, 'mà': 49, 'nên': 50, 'con': 51, 'nhiều': 52, 'trong': 53, '1': 54, 'game': 55, 'từng': 56, 'cùng': 57, 'vào': 58, 'giờ': 59, 'để': 60, 'nhà': 61, 'gái': 62, 'phải': 63, 'chở': 64, 'mập': 65, 'đồng': 66, 'từ': 67, 'quá': 68, 'xem': 69, 'phim': 70, 'tới': 71, 'nói': 72, 'như': 73, 'vô': 74, 'đẹp': 75, 'trai': 76, 'muốn': 77, 'uống': 78, 'hôm': 79, 'ơi': 80, 'cua': 81, 'anh': 82, 'hỏi': 83, 'hết': 84, 'ế': 85, 'nghĩ': 86, 'bản': 87, 'tay': 88, 'hã': 89, 'xe': 90, 'thiện':

In [15]:
print(sequences)
print(padded_sequences)

[[2, 9, 10, 3], [2, 9, 69, 70, 188, 189], [27, 10, 190, 106, 46, 3], [5, 107, 191, 11, 10, 3], [53, 2, 192, 193, 194, 195, 28], [10, 9, 2, 108, 35, 20, 22, 28, 10, 196], [197, 198, 10, 69, 31, 109, 18, 22, 28, 10, 199, 71], [2, 4, 9, 8, 110, 11, 17, 3], [200, 201, 72, 54, 111, 202, 203], [2, 8, 17, 6], [8, 4, 112, 3], [2, 47, 113, 6], [2, 4, 9, 114, 115, 3], [2, 8, 17, 6], [8, 4, 112, 3], [2, 47, 113, 6], [2, 4, 9, 114, 115, 3], [2, 48, 116, 21, 12], [2, 8, 17, 6], [2, 9, 20, 204, 3], [2, 36, 205, 206, 207, 208, 3], [2, 4, 9, 209, 3], [2, 4, 210, 23, 55, 3], [21, 2, 18, 22, 73, 21, 12], [19, 8, 17, 6], [2, 4, 56, 211, 212, 213, 214, 12, 3], [2, 4, 36, 215, 216, 217, 74, 57, 218, 42, 74, 57, 58, 43, 42, 74, 57, 3], [2, 46, 5, 219, 220, 6], [13, 2, 75, 76, 15, 49, 117, 6, 4, 22, 28, 21], [2, 4, 37, 59, 47, 53, 118, 3], [2, 4, 221, 222, 37, 59, 6], [119, 223, 224, 225, 226, 227], [119, 5, 228, 6], [2, 4, 38, 229, 230, 6], [2, 4, 231, 5, 23, 58, 232, 120, 3], [10, 50, 39, 121, 233, 12, 60,

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(LSTM(150))
model.add(Dense(16, activation='relu')) # 16 class to categorize data
model.add(Dense(16, activation='relu')) # 2 times, more exactly
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            32000     
_________________________________________________________________
conv1d (Conv1D)              (None, 20, 32)            1568      
_________________________________________________________________
lstm (LSTM)                  (None, 150)               109800    
_________________________________________________________________
dense (Dense)                (None, 16)                2416      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 206)               3502      
Total params: 149,558
Trainable params: 149,558
Non-trainable params: 0
__________________________________________________

In [11]:
epochs = 400
history = model.fit(padded_sequences, np.array(labels), epochs=epochs)

Epoch 1/400
7/7 [==============================] - 5s 24ms/step - loss: 5.3289 - accuracy: 0.0000e+00
Epoch 2/400
7/7 [==============================] - 0s 25ms/step - loss: 5.3280 - accuracy: 0.0000e+00
Epoch 3/400
7/7 [==============================] - 0s 25ms/step - loss: 5.3278 - accuracy: 0.0000e+00
Epoch 4/400
7/7 [==============================] - 0s 25ms/step - loss: 5.3272 - accuracy: 0.0069
Epoch 5/400
7/7 [==============================] - 0s 29ms/step - loss: 5.3265 - accuracy: 0.0108
Epoch 6/400
7/7 [==============================] - 0s 27ms/step - loss: 5.3259 - accuracy: 0.0069
Epoch 7/400
7/7 [==============================] - 0s 31ms/step - loss: 5.3244 - accuracy: 0.0036
Epoch 8/400
7/7 [==============================] - 0s 50ms/step - loss: 5.3170 - accuracy: 0.0144
Epoch 9/400
7/7 [==============================] - 0s 25ms/step - loss: 5.3116 - accuracy: 0.0087
Epoch 10/400
7/7 [==============================] - 0s 26ms/step - loss: 5.2952 - accuracy: 0.0019
Epoch 1

7/7 [==============================] - 0s 35ms/step - loss: 1.1654 - accuracy: 0.6817
Epoch 84/400
7/7 [==============================] - 0s 35ms/step - loss: 1.1431 - accuracy: 0.6560
Epoch 85/400
7/7 [==============================] - 0s 31ms/step - loss: 1.1665 - accuracy: 0.6621
Epoch 86/400
7/7 [==============================] - 0s 30ms/step - loss: 1.1182 - accuracy: 0.6859
Epoch 87/400
7/7 [==============================] - 0s 29ms/step - loss: 1.0940 - accuracy: 0.6750
Epoch 88/400
7/7 [==============================] - 0s 28ms/step - loss: 1.0537 - accuracy: 0.7400
Epoch 89/400
7/7 [==============================] - 0s 49ms/step - loss: 0.9849 - accuracy: 0.7412
Epoch 90/400
7/7 [==============================] - 0s 60ms/step - loss: 1.0783 - accuracy: 0.6954
Epoch 91/400
7/7 [==============================] - 0s 46ms/step - loss: 1.0458 - accuracy: 0.6925
Epoch 92/400
7/7 [==============================] - 0s 34ms/step - loss: 0.9741 - accuracy: 0.7500
Epoch 93/400
7/7 [=====

7/7 [==============================] - 0s 28ms/step - loss: 0.5850 - accuracy: 0.7856
Epoch 165/400
7/7 [==============================] - 0s 28ms/step - loss: 0.5519 - accuracy: 0.7706
Epoch 166/400
7/7 [==============================] - 0s 28ms/step - loss: 0.5131 - accuracy: 0.8062
Epoch 167/400
7/7 [==============================] - 0s 28ms/step - loss: 0.5180 - accuracy: 0.8040
Epoch 168/400
7/7 [==============================] - 0s 27ms/step - loss: 0.5065 - accuracy: 0.7696
Epoch 169/400
7/7 [==============================] - 0s 27ms/step - loss: 0.4774 - accuracy: 0.7858
Epoch 170/400
7/7 [==============================] - 0s 27ms/step - loss: 0.4345 - accuracy: 0.7866
Epoch 171/400
7/7 [==============================] - 0s 29ms/step - loss: 0.4950 - accuracy: 0.7801
Epoch 172/400
7/7 [==============================] - 0s 27ms/step - loss: 0.4816 - accuracy: 0.7836
Epoch 173/400
7/7 [==============================] - 0s 28ms/step - loss: 0.4622 - accuracy: 0.7783
Epoch 174/400


7/7 [==============================] - 0s 26ms/step - loss: 0.3272 - accuracy: 0.8136
Epoch 246/400
7/7 [==============================] - 0s 24ms/step - loss: 0.3513 - accuracy: 0.8074
Epoch 247/400
7/7 [==============================] - 0s 25ms/step - loss: 0.3973 - accuracy: 0.7592
Epoch 248/400
7/7 [==============================] - 0s 26ms/step - loss: 0.3738 - accuracy: 0.8047
Epoch 249/400
7/7 [==============================] - 0s 29ms/step - loss: 0.3573 - accuracy: 0.7885
Epoch 250/400
7/7 [==============================] - 0s 31ms/step - loss: 0.4009 - accuracy: 0.7893
Epoch 251/400
7/7 [==============================] - 0s 26ms/step - loss: 0.3565 - accuracy: 0.8047
Epoch 252/400
7/7 [==============================] - 0s 26ms/step - loss: 0.3496 - accuracy: 0.7923
Epoch 253/400
7/7 [==============================] - 0s 24ms/step - loss: 0.4065 - accuracy: 0.7789
Epoch 254/400
7/7 [==============================] - 0s 27ms/step - loss: 0.3542 - accuracy: 0.7770
Epoch 255/400


7/7 [==============================] - 0s 25ms/step - loss: 0.3314 - accuracy: 0.7993
Epoch 327/400
7/7 [==============================] - 0s 29ms/step - loss: 0.3617 - accuracy: 0.7636
Epoch 328/400
7/7 [==============================] - 0s 32ms/step - loss: 0.3513 - accuracy: 0.8009
Epoch 329/400
7/7 [==============================] - 0s 31ms/step - loss: 0.3457 - accuracy: 0.7930
Epoch 330/400
7/7 [==============================] - 0s 29ms/step - loss: 0.3380 - accuracy: 0.8021
Epoch 331/400
7/7 [==============================] - 0s 28ms/step - loss: 0.3935 - accuracy: 0.7809
Epoch 332/400
7/7 [==============================] - 0s 28ms/step - loss: 0.3845 - accuracy: 0.7816
Epoch 333/400
7/7 [==============================] - 0s 27ms/step - loss: 0.3754 - accuracy: 0.7642
Epoch 334/400
7/7 [==============================] - 0s 23ms/step - loss: 0.3393 - accuracy: 0.8035
Epoch 335/400
7/7 [==============================] - 0s 23ms/step - loss: 0.3467 - accuracy: 0.7785
Epoch 336/400


In [12]:
# saving model
model.save("chat_model.h5")

# implements binary protocols for serializing and de-serializing a Python object structure.
import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
import random
from difflib import SequenceMatcher

def chat(inp):
    # load trained model
    model = keras.models.load_model('chat_model.h5')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                         truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])
    new_tag = tag.astype(np.int)
    for i in range(len(df)):
        if i == new_tag:
            print("ChatBot:", df['answers'][i])
            print(df['questions'][i], SequenceMatcher(None, df['questions'][i], inp).ratio(), new_tag, "\n")

In [14]:
while(True):
    inp=str(input('Nhập câu: '))
    if(inp.lower() == 'thoat' or inp.lower() == 'thoát'):
        break
    chat(inp)

Nhập câu: thoat
